In [ ]:
import numpy as np
import matplotlib.pyplot as plt # pour les plots
from scipy.special import gamma # pour la fonction gamma d'Euler

# TP: Simulation de variables aléatoires
Auteur: Noé Blassel

## Cours
On a vu au cours des chapitres précédents des méthodes pour calculer la loi de diverses variables aléatoires obtenues comme des transformations de variables aléatoires plus simples dont la densité est connue analytiquement. Dans de nombreux cas, il est possible d'établir des relations entre diverses lois, ce qui permet à la fois de faire des calculs analytiques, et de tirer des échantillons d'une loi "compliquée" à partir d'échantillons d'une loi "simple".

**e.g.** On a vu que $$ X_1 + X_2 + \dots + X_n \sim \Gamma(N,\lambda),\qquad (X_k)_{1\leq k\leq n}\text{ iid }\mathcal E(\lambda).$$

Ci-dessous, quelques unes des relations analytiques entre différentes familles de lois de probabilités sur $\R$.
<details>
<summary>
<b>Cliquez pour dévoiler.</b>
</summary>

<img src="https://noeblassel.github.io/teaching/relations.jpg" alt="Relations entre lois usuelles" width="80%" />

</details>

Cependant, si la transformation est trop complexe, il est souvent nécessaire d'avoir recours à la simulation.

__Exemple__ 

On veut connaître l'influence d'un trait $T$ sur le taux de reproduction d'une certaine espèce. Autrement dit, on veut calculer, pour différents $t$, l'espérance
$$R(t)=\mathbb E[N(t,X)],$$
où $t$ représente la valeur du trait $t$ d'un individu qu'on fera varier, $N$ le nombre de ses descendants directs, et qui dépend à la fois du trait $t$ et d'une variable aléatoire $X$ exprimant l'interaction entre l'individu et son environnement. Pour des modèles réalistes, les interactions sont bien trop complexes pour que la loi de $X$ soit explicite analytiquement: la seule solution est de simuler l'environnement, pour estimer $R(t)$, par exemple en utilisant la loi des grands nombres.

Dans de nombreuses situations (finance, physique statistique, biologie, machine learning...), les systèmes en jeu sont trop complexes pour être analysés explicitement, et on a donc souvent recours à des méthodes stochastiques de simulation pour estimer diverses quantités d'intérêt.

Dans ce TP, on va voir plusieurs méthodes de base pour simuler des variables aléatoires. 

### I) Génération de nombres pseudo-aléatoires (PRNG)
La plupart des ordinateurs ne permettent pas de générer des nombres véritablement aléatoires. On a plutôt recours à des méthodes dites "pseudo-aléatoires". Une méthode simple, étant donné $N$ (généralement, $N=2^{16},2^{32}$ ou $2^{64}$) est basée sur l'itération suivante, dite du __LCG (Linear congruence generator)__:
$$X_{n+1} = a X_{n} +b\mod N.$$
où on choisit les entiers $a,b\in \N$ avec précaution, pour s'assurer
- Que la suite $(X_n)_{n\geq 1}$ soit aussi <i>uniformément</i> répartie que possible dans $\{0,\dots,N-1\}$.
- Que la période de cette suite ($\leq N$) soit aussi grande que possible.
- Que la valeur de $X_n$ soit aussi <i>indépendante</i> que possible de celles des $(X_{k})_{k\leq n_1}$.

L'étude théorique de ces propriétés est encore un sujet actif de recherche, à l'intersection de plusieurs domaines des mathématiques et de l'informatique.

Le comportement de cette suite est néanmoins déterministe: elle est entièrement déterminée par la valeur de la graine $X_0$, ou __random seed__. Les notions d'uniformité et d'indépendance en jeu sont donc différentes de celles étudiées dans le domaine des probabilités.

En pratique, on a recours a des variantes plus sophistiquées, telles que le [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) ou le [Xorshift/Xoshiro](https://en.wikipedia.org/wiki/Xorshift).

On considère désormais qu'on dispose de variables aléatoires **iid** $\mathcal U(0,1)$, données en pratique par la suite $(X_n/N)_{n\geq 1}$.

Dans `numpy`, il est possible de spécifier le générateur utilisé, ainsi que la seed, avec la fonction [`np.random.seed`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.seed.html). Cette possibilité est très importante en pratique, car elle permet d'obtenir des résultats __reproductibles__.

Il est aussi possible d'intéragir avec/de modifier le PRNG, mais nous ne nous aventurerons pas là-dedans aujourd'hui.

Pour tirer des variables **iid** uniformes sur $[0,1]$, on peut utiliser les fonctions [`np.random.rand`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html) ou [`np.random.uniform`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html).


In [ ]:
seed = 17122024
np.random.seed(seed)
print(np.random.rand(2,3))

Si vous relancez le notebook, vous devriez obtenir le même résultat.
Aujourd'hui, on va se concentrer sur le cas des variables aléatoires à densités sur $\R$. On va voir essentiellement deux méthodes: la méthode d'inversion de la fonction de répartition, et la méthode du rejet.

### II) Inversion de la fonction de répartition

#### Proposition
Soit $X$ une variable aléatoire suivant la densité $p$. On suppose que pour certains $-\infty\leq a < b\leq +\infty$, on a $\mathbb P(a<X<b) = 1$, et que la fonction de répartition $F_X$ de $X$ est strictement croissante et continue sur $(a,b)$. Alors la variable aléatoire
$$ \widetilde X = F_X^{-1}(U),\qquad U\sim\mathcal U(0,1)$$
a la même loi que $X$.

<details>
<summary>
<b>Preuve</b>
</summary>

$F_X$ étant strictement croissante et continue, son inverse est bien définie sur $(a,b)$ et mesurable, donc $F_X^{-1}(U)$ est une variable aléatoire.
De plus, on a $F_X(x)\xrightarrow{x\to a}0$ et $F_X(x)\xrightarrow{x\to b}1$. Donc $F_X^{-1}: (a,b)\to (0,1)$.
 D'autre part, pour $a < x < b$, on a 

$$ \mathbb P(F_X^{-1}(U) \leq x) = \mathbb P(U \leq F_X(x)) = F_X(x) = \mathbb P(X\leq x).$$
</details>

__Exemples__
- $a\tan(\pi U)$ suit une loi de Cauchy $\mathcal C(a)$ pour $a>0$.
- $-\frac1{\lambda} \ln U$ suit une loi exponentielle $\mathcal E(\lambda)$ pour $\lambda>0$.

Pour montrer ces relations, on inverse les fonctions de répartitions des lois correspondantes:
- Pour $C\sim \mathcal C(a)$, on a $F_C(x) = \frac1\pi \arctan \frac{x}{a} + \frac12$, d'où $F_C^{-1}(u) =  a\tan\left(\pi(u-\frac12)\right)$. Par $\pi$-periodicité de $\tan$, on vérifie facilement que $\tan(\pi(U-\frac12)) \overset{\mathcal L}{=} \tan(\pi U)$ (utiliser la méthode de la fonction muette).
- Pour $E\sim \mathcal E(\lambda)$, on a $F_E(x) = 1 - \mathrm{e}^{-\lambda x}$, d'où $F_E^{-1}(u) = -\frac{1}{\lambda}\ln(1-u)$. On a de plus $1-U\overset{\mathcal L}{=} U$, d'où $F_E^{-1}(U) \overset{\mathcal L}{=} F_E^{-1}(1-U)$.

#### Vérification empirique
On utilise les fonctions [`np.random.standard_cauchy`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.standard_cauchy.html) et [`np.random.exponential`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.exponential.html) pour tirer les échantillons de référence. 

Pour tracer des histogrammes, on utilise la fonction [`plt.hist`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html).

In [ ]:
N = 100000 # nombres d'échantillons
lam = 2 # lambda

U = np.random.rand(N) # variables iid uniformes sur (0,1)
C = np.random.standard_cauchy(N)
E = np.random.exponential(scale=1/lam,size=N) # attention, scale est l'inverse du lambda

Ctilde = np.tan(np.pi * U)
Etilde = -np.log(U)/lam

In [ ]:
fig = plt.figure()
plt.hist(C, density=True, label="Cauchy (numpy)",bins=50, alpha=1/3, color="r",range=(-10,10))
plt.hist(Ctilde, density=True, label="Cauchy (maison)",bins=50, alpha=1/3, color="b",range=(-10,10))
plt.legend(loc="best")
plt.show()

In [ ]:
fig = plt.figure()
plt.hist(E, density=True, label="Expo (numpy)",bins=50, alpha=1/3, color="r")
plt.hist(Etilde, density=True, label="Expo (maison)",bins=50, alpha=1/3, color="b")
plt.legend(loc="best")
plt.show()

Souvent, il n'est pas possible de calculer ou d'inverser la fonction de répartition (pour des variables aléatoires réelles).
 On peut alors avoir recours à la méthode de rejet, qui s'applique directement pour le cas de vecteurs aléatoires à valeurs dans $\R^d$.

### III) Méthode de rejet
La méthode s'applique 
On suppose qu'on veut simuler une variable aléatoire $X$ suivant une densité de probabilités $p:\mathcal O\to\R_+$ supposée connue, avec $\mathcal O\subset \R^d$ (i.e. $\mathbb P(X\in\mathcal O)=1$). Au lieu d'échantillonner directement cette densité, on suppose qu'on dispose d'une densité $q:\mathcal O\to\R^*_+$ __strictement positive__ sur $\mathcal O$ et d'une constante $k>0$ telle que 
$$p(x) \leq k q(x)$$
pour presque tout $x\in \mathcal O$. La densité $q$ est typiquement simple a échantillonner, avec des méthodes classiques. L'algorithme de la méthode de rejet est le suivant:
- Tirer $Z\sim q$, et $U\sim \mathcal U(0,1)$ indépendante de $Z$.
- Si $kq(Z)U \leq p(Z)$, on utilise $Y$ comme échantillon pour la densité $p$.
- Sinon, on rejette cet échantillon et on recommence.

Illustrations (empruntées [ici](https://gregorygundersen.com/blog/2019/09/01/sampling/))

<img src="https://gregorygundersen.com/image/sampling/rejection_sampling_diagram.png" alt="Méthode de rejet" width="40%" />
<img src="https://gregorygundersen.com/image/sampling/rejection_sampling_example.png" alt="Méthode de rejet" width="60%" />

Plus formellement, prenons $(Z_n)_{n\geq 1}$ et $(U_n)_{n\geq 1}$ des suites indépendantes, chacune <i>iid</i>, avec $Z_1$ admettant la densité $q$ et $U_1$ étant uniforme sur $(0,1)$. Notons $N$ l'indice du premier échantillon retenu
$$N = \inf\{n\geq 1:\, kq(Z_n)U_n\leq p(Z_n)\}.$$
#### Théorème
$Z_N$ admet la densité de probabilité $p$, et $N$ suit une loi géométrique de paramètre $1/k$, indépendante de $(Z_N,kq(Z_N)U_N)$.

<details>
<summary>
<b> Preuve </b>
</summary>

On calcule la loi de $(Z_1,kq(Z_1)U_1)$. Soit $f:\R^d\times \R \to \R$ mesurable bornée. On a, par le théorème de transfert
$$ \begin{aligned}\mathbb E\left[f(Z_1,kq(Z_1)U_1)\right] &= \int_{\R^d}\int_0^1 f(z,kq(z)u)q(z)\mathrm{d} u\,\mathrm{d} z,\\
&=\int_{\mathcal O}\int_0^{kq(z)} f(z,v) q(z)/(k q(z))\,\mathrm{d} v\,\mathrm{d}z,\\
&= \frac1{k}\int_{\mathcal O\times \R} f(z,v){\bf{1}}_{0\leq v \leq kq(z)}\,\mathrm{d}v\,\mathrm{d}z,
\end{aligned}
$$
en utilisant le changement de variable $v = kq(z)u$ de $(0,1)$ vers $(0,kq(z))$ et le fait que $kq>0$ sur $\mathcal O$. Il s'ensuit que $(Z_1,kq(Z_1)U_1)$ est uniformément réparti sous le graphe de $kq$:
$$(Z_1,kq(Z_1)U_1) \sim \mathcal U(D_{k,q}),\qquad D_{k,q} = \{(z,v)\in \mathcal O \times \R:\,v\leq kq(z)\},\qquad |D_{k,q}|=k.$$
Calculons la probabilité de retenir $Z_1$, qui est donnée par
$$\mathbb P(N=1) = \mathbb P(kq(Z_1)U_1 \leq p(Z_1)) = \frac1k\int_{\mathcal O\times \R} {\bf{1}}_{v\leq p(z)} {\bf{1}}_{0\leq v \leq kq(z)}\,\mathrm{d} v\,\mathrm{d}z = \frac1k\int_{\mathcal O}\int_{0}^{p(z)}\,\mathrm{d} v\,\mathrm{d} z = \frac1k\int_{\mathcal O}p(z)\,\mathrm{d} z = \frac1k,$$
où on a utilisé que $p(z)<kq(z)$ pour presque tout $z\in \mathcal O$. Donc $N$, étant le rang d'atteinte de $1$ pour suite de variables de Bernoulli <i>iid</i> de paramètre $p=\frac1k$, suit une loi géométrique de paramètre $p=\frac1k$.

Pour calculer la loi de $(N,Z_N,kq(Z_N)U_N)$, on se donne $g:\N\to \R$ et $f:\mathcal O\times \R\to \R$ mesurables bornées. Alors
$$ 
\begin{aligned}
\mathbb{E}\left[g(N)f(Z_N,kq(Z_N)U_N)\right] &= \mathbb{E}\left[\sum_{n\geq 1} {\bf{1}}_{N=n} g(n) f(z_n,kq(z_n)u_n)\right]\\
&=\sum_{n\geq 1}g(n)\mathbb{E}\left[{\bf{1}}_{ kq(Z_1)U_1 > p(Z_1)}\dots{\bf{1}}_{kq(Z_{n-1})U_{n-1}>p(Z_{n-1})}{\bf{1}}_{kq(Z_n)U_n\leq p(Z_n)}f(Z_n,kq(Z_n)U_n)\right]\\
&=\sum_{n\geq 1}g(n)\mathbb{P}\left(kq(Z_1)U_1>p(Z_1)\right)^{n-1}\mathbb{E}\left[{\bf{1}}_{kq(Z_n)U_n\leq p(Z_n)}f(Z_n,kq(Z_n)U_n)\right]\\
&=\sum_{n\geq 1}g(n)\left(1-\frac1k\right)^{n-1}\frac1k\int_{\mathcal O\times \R}f(z,v){\bf{1}}_{v\leq p(z)}{\bf{1}}_{0\leq v\leq kq(z)}\\
&= \left[\sum_{n\geq 1}g(n)\frac1k\left(1-\frac 1k\right)^{n-1}\right] \left[\int_{\mathcal O\times \R} {\bf{1}}_{0\leq v\leq p(z)} f(z,v)\,\mathrm{d} v\,\mathrm{d} z\right]\\
&= \mathbb{E}[g(N)]\mathbb{E}[f(Z,W)],
\end{aligned}
$$
où $(Z,W)$ est uniformément répartie sous le graphe de $p$:
$$(Z,W)\sim \mathcal U(D),\qquad D = \{(z,w)\in \mathcal O \times \R:\, w\leq p(z)\}.$$
Ce calcul montre que $N$ est indépendante de $(Z_N,kq(Z_N)U_N)$, et que $(Z_N,kq(Z_N)U_N)\overset{\mathcal L}{=} (Z,W)$. En particulier, $Z_N$ admet la densité

$$z\mapsto \int_{\R} {\bf{1}}_{0\leq w\leq p(z)}\,\mathrm{d}w = p(z)$$
par la formule des densités marginales.
</details>

Il est clair qu'il faut choisir $k$ aussi petit que possible, ce qui donne $k^* = \sup_{z\in\mathcal O} p(z)/q(z)$. Il faut choisir $q$ de manière à ce que $k^*$ soit petit. Ceci est d'autant plus vrai que $q$ a la même "forme" que $p$ (pour $q=p$, $k^*=1$ et on accepte tous les échantillons).

#### Vérification empirique

On va simuler $Y\sim \Gamma(3/2,1)$ a l'aide d'une $X\sim \mathcal E(\lambda)$. On a donc
$$p(x) = C\sqrt{x}\mathrm{e}^{-x}{\bf{1}}_{x>0},\qquad q_{\lambda}(x) = \lambda\mathrm{e}^{-\lambda x}{\bf{1}}_{x>0},$$
où $C=\Gamma(3/2)^{-1}=2/\sqrt\pi$. On a
$$F(x,\lambda) := p(x)/q_{\lambda}(x) = C \lambda^{-1}\mathrm{e}^{(\lambda-1)x}\sqrt{x}\qquad \text{sur }\R_+^*.$$

Calculons d'abord $k^*(\lambda) = \sup_{x} p(x)/q_\lambda(x)$. Pour $\lambda\geq 1$, $k^*(\lambda)=+\infty$. Pour $\lambda\in(0,1)$, on calcule
$$\partial_x F(x,\lambda)= C \lambda^{-1} \mathrm{e}^{(\lambda-1)x}\left(\frac1{2\sqrt x} + (\lambda-1)\sqrt{x}\right),$$
donc $\partial_x F(x,\lambda)$ admet un point critique en $\frac1{2(1-\lambda)}$, qui est en fait un maximum, par une étude des variations de $x\mapsto F(x,\lambda)$.
On a donc
$$k^*(\lambda) = F(1/(2(1-\lambda)),\lambda) = C \lambda^{-1}\exp\left(\frac{\lambda-1}{2(1-\lambda)}\right)/\sqrt{2(1-\lambda)} = C\mathrm{e}^{-\frac12}/\sqrt{2\lambda^2(1-\lambda)},$$
qui est minimisé pour $\lambda^2(1-\lambda)$ maximal, i.e. pour $\lambda=2/3$.

On compare $\lambda = 1/3$ et $\lambda = 2/3$.

In [ ]:
lam1 = 1/3
lam2 = 2/3

T = np.linspace(0,10,1000) # points pour les figures
C = 2/np.sqrt(np.pi)

p = lambda x: C * np.sqrt(x) * np.exp(-x) # densité p
q1 = lambda x: lam1 * np.exp(-lam1 * x) # densité d'une expo de parametre lambda1
q2 = lambda x: lam2 * np.exp(-lam2 * x) # idem pour lambda 2

k1 = C * np.exp(-0.5) / np.sqrt(2 * (lam1 ** 2) * (1-lam1)) # constante k pour q1
k2 = C * np.exp(-0.5) / np.sqrt(2 * (lam2 ** 2) * (1-lam2)) # constante k pour q2

print("Calcul des proportions théoriques d'échantillons retenus: 1/k(0.5) = ",1/k1,", 1/k(2/3) = ",1/k2)

fig = plt.figure()
plt.plot(T,p(T),label="p",color="r")
plt.plot(T,k1*q1(T),label="k1*q1",color="b")
plt.plot(T,k2*q2(T),label="k2*q2",color="g")
plt.legend(loc="best")

plt.show()

Pour effectuer le rejet, on peut utiliser la syntaxe d'indexation booléenne de `numpy`: [`X[Y < Z]`](https://numpy.org/doc/stable/user/basics.indexing.html#boolean-array-indexing).

In [ ]:
N = 10000 # nombres d'échantillons
U = np.random.rand(N) # variables uniformes pour le rejet

E1 = -np.log(np.random.rand(N))/lam1 # exponentielles iid de parametre lambda1
E2 = -np.log(np.random.rand(N))/lam2 # de parametre lambda 2

Y1 = E1[k1*q1(E1)*U <= p(E1)] # méthode de rejet
Y2 = E2[k2*q2(E2)*U <= p(E2)]

print("Proportion empirique d'échantillons retenus : pour lambda = 1/3:" ,len(Y1)/N,", et  pour lambda = 2/3: ",len(Y2)/N,".")

fig = plt.figure()
plt.hist(Y1, density=True, label="lambda = 1/3",bins=50, alpha=1/3, color="b",range=(0,10))
plt.hist(Y2,density=True, label="lambda = 2/3",bins=50, alpha=1/3, color="g",range=(0,10))
plt.plot(T,p(T),color="r",label="p")
plt.legend(loc="best")
plt.show()

### IV) Méthode de Box-Muller

On termine ce cours avec une méthode pour simuler des variables gaussiennes <i>iid</i> $\mathcal N(0,1)$, la méthode de Box-Muller.


#### Proposition
Soit $R\sim \mathcal E(1/2)$ et $\Theta \sim \mathcal U(0,2\pi)$ indépendantes. Alors
$$ (X,Y) = \sqrt{R}(\cos \Theta,\sin \Theta)$$
est un vecteur gaussien de covariance $\mathrm{Id}_2$.

<details>
<summary>
<b> Preuve </b>
</summary>

Le changement de variables
$\varphi(r,\theta)=(\sqrt{r}\cos\theta,\sqrt{r}\sin\theta)$ est un $\mathcal C^1$ difféomorphisme de $\R_+^*\times (0,2\pi)$ dans $\mathcal O = \R^2 \setminus \{(x,y)\in \R^2:\, y=0,\,x\geq 0\}.$
Son jacobien est donné par
$$ \det \begin{bmatrix} \cos\theta/(2\sqrt r) & \sin\theta/(2\sqrt r)\\ -\sqrt{r}\sin\theta & \sqrt{r}\cos\theta\end{bmatrix} = \frac12.$$
La densité jointe du couple $(R,\Theta)$ est donnée par
$$p(r,\theta) = \frac{1}{4\pi} \mathrm{e}^{-r/2}{\bf{1}}_{r>0}{\bf 1_{0<\theta<2\pi}}.$$
Par changement de densité (en notant que $R = X^2 + Y^2$), $(X,Y)$  admet la densité:
$$q(x,y) = \frac1{2\pi}\mathrm{e}^{-\frac{x^2+y^2}{2}}{\bf{1}}_{(x,y)\in\mathcal O} = \left(\frac{1}{\sqrt{2\pi}}\mathrm{e}^{-\frac{x^2}2}\right) \left(\frac{1}{\sqrt{2\pi}}\mathrm{e}^{-\frac{y^2}2}\right)$$
presque partout, et on reconnaît la densité du vecteur gaussien annoncé.

</details>


#### Vérification empirique

Avec `numpy`, on peut échantilloner des variables normales centrées réduites à l'aide des fonctions [`np.random.standard_normal`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.standard_normal.html) et [`np.random.normal`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html).

In [ ]:
N = 10000

G = np.random.standard_normal(N)
lam = 0.5
R = -np.log(np.random.rand(N))/lam # exponentielles de parametre 0.5
Theta = np.random.rand(N) * (2*np.pi) # uniformes sur [0,2pi]

X = np.sqrt(R) * np.cos(Theta)
Y = np.sqrt(R) * np .sin(Theta)

T = np.linspace(-3,3,1000) # points pour le plot

fig = plt.figure()
plt.hist(X, density=True, label="X",bins=50, alpha=1/3, color="r",range=(-3,3))
plt.hist(Y,density=True, label="Y",bins=50, alpha=1/3, color="b",range=(-3,3))
plt.plot(T,np.exp(-T**2/2)/np.sqrt(2*np.pi),color="black",label="p")
plt.legend(loc="best")
plt.show()

Calculons la covariance empirique entre `X` et `Y`. Notons $G_n = (X_n,Y_n)^\top = (\sqrt{R_n}\cos\Theta_n,\sqrt{R_n}\sin\Theta_n)^\top$. Celle-ci est donnée (on sait a priori que les variables sont centrées) par

$$C_N = \frac1N\sum_{n=1}^N G_n G_n^\top.$$
Par la la loi forte des grands nombres, cette matrice converge presque sûrement vers la matrice identité $\mathrm{Id}_2$. Dans la cellule ci-dessous, on utilise deux fonctions
[`np.column_stack`](https://numpy.org/doc/stable/reference/generated/numpy.column_stack.html) qui transforme deux vecteurs 1D en les colonnes d'une matrice, et la fonction [`np.einsum`](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html), qui permet (entre autres) de calculer efficacement la covariance empirique.

In [ ]:
G = np.column_stack([X,Y])
C = np.einsum("ni,nj",G,G)/N
print(C.shape)
print(C)

## Travaux pratiques

### V) Visualisation de la loi des grands nombres et du TCL


#### 1) Loi des grands nombres

Simuler, par la méthode de votre choix, $N$ variables $(X_n)_{1\leq n\leq N}$ <i>iid</i> de loi $\mathcal E(1/2)$. Calculer la moyenne empirique
$$M_n = \frac1n \sum_{k=1}^n X_k$$
en fonction de $n$ et tracer le résultat. Qu'observe-t-on ?

On pourra utiliser la fonction [`np.cumsum`](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html) qui renvoie le vecteur formé des sommes partielles de son argument.

In [ ]:
N = 10000 # nombres d'échantillons

## Echantillonner les X_n
# X = ...

## Calculer les moyennes empiriques
N_range = np.linspace(1,N,N) # [1.0,2.0,..., N]
# M = ...

## plot 
# fig = plt.figure()
# plt.plot(N,M,label = "moyenne empirique")
# plt.show()

#### 2) Echec de la loi des grands nombres
Faites la même chose avec des variables de Cauchy $\mathcal C(1)$ $(Y_n)_{1\leq n\leq N}$. Qu'observe-t-on ?

In [ ]:
## Echantilloner les Y_n

## Calculer les moyennes empiriques

## plot

# fig = plt.figure()
# plt.plot(N,M,label = "moyenne empirique")
# plt.show()

#### 3) Visualisation du Théorème central limite

On note cette fois
$$Z_N = \frac{\sqrt{N}}{\sigma}\left(\frac{1}{N}\sum_{k=1}^N X_k -\mu\right),$$
où $\sigma^2 = \mathrm{Var}(X_1)$ et $\mu = \mathbb{E}[X_1]$.

Quel est le comportement asymptotique de $Z_N$ quand $N\to\infty$? Simuler $M$ échantillons <i>iid</i> de $Z_N$. On pourra se servir de la fonction [`np.mean`](), avec la syntaxe `np.mean(X,axis=0)` pour prendre la moyenne selon une seule dimension d'une certaine matrice aléatoire `X`.

####

In [ ]:
N = 30
M = 10000 # nombre d'échantillons de Z
lam = 1/2
E = -np.log(np.random.rand(N,M))/lam


# Calculer Z
# sigma = 
# mu = 
# Z = 

# plot

# T = np.linspace(-3,3,1000)
# fig = plt.figure()
# plt.plot(T,np.exp(-T**2/2)/np.sqrt(2*np.pi),color="black")
# plt.hist(Z,density=True,bins=50,label = "Z_N", range=(-3,3),color="r",alpha=0.3)
# plt.legend(loc="best")
# plt.show()

#### 4) Échec du Théorème central limite

Faire de même avec des variables de Cauchy. On prendra $\mu=0$ et $\sigma = 1$.


In [ ]:
N = 120
M = 10000 # nombre d'échantillons de Z
lam = 1/2
C = np.random.standard_cauchy(size=(N,M))


## Calculer Z
# mu = 0
# sigma = 1
# Z = 

## plot 
# T = np.linspace(-3,3,1000)
# fig = plt.figure()
# plt.plot(T,np.exp(-T**2/2)/np.sqrt(2*np.pi),color="black")
# plt.hist(Z,density=True,bins=50,label = "Z_N", range=(-3,3),color="r",alpha=0.3)
# plt.legend(loc="best")
# plt.show()


En fait, en calculant la fonction caractéristique, on peut montrer que 
$$C_N = \frac1{N}\sum_{n=1}^N Y_n \sim \mathcal C(1).$$
Ceci implique que ni la loi des grands nombres, ni le TCL ne peuvent être vérifiés.

In [ ]:
N = 1 # jouer avec le N
M = 10000
C = np.random.standard_cauchy(size=(N,M))

# plot 
T = np.linspace(-10,10,1000)
fig = plt.figure()
plt.plot(T,1.0/(np.pi*(1+T**2)),color="black",label="densité Cauchy(1)")
plt.hist(np.mean(C,axis=0),density=True,bins=50,label = "Histogramme de C(N)", range=(-10,10),color="r",alpha=0.3)
plt.legend(loc="best")
plt.show()

### VI) Simulation efficace d'une variable géométrique
On note $\lfloor x\rfloor$ la partie entière de $x\in \R$. Soit $X\sim \mathcal E(\lambda)$ avec $\lambda>0$. Quelle est la loi de $1+\lfloor X\rfloor$? Étant donné $p\in(0,1)$, en déduire un $\lambda>0$ tel que

$$1 + \lfloor - \frac1\lambda\ln U\rfloor \sim \mathcal{Geo}(p).$$

Implémenter la méthode ci-dessus. On peut la vérifier en utilisant la fonction [`np.random.geometric`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.geometric.html).

In [ ]:
N = 1000 
p = 1/3

G1 = np.random.geometric(p,N)
lam = 0.1 # changer ici
U = np.random.rand(N)
G = 1 + np.floor(-np.log(U) / lam)

On fournit du code pour comparer les deux lois obtenues.

In [ ]:
max_k = 20
K_range = np.linspace(1,max_k,max_k).reshape(1,max_k)

P1 = (G1.reshape(N,1) == K_range).mean(axis=0) # loi empirique de G1
P2 = (G.reshape(N,1) == K_range).mean(axis=0) # loi empirique de G2

fig = plt.figure()
plt.scatter(K_range,P1,marker="x",color="r",alpha=0.5,label="numpy")
plt.scatter(K_range,P2,marker="x",color="b",alpha=0.5,label="maison")
plt.legend(loc="best")
plt.show()

### VII) Simulation d'une loi $\beta$ par rejet

#### Question 1)
Montrer que si
$$ D = \left\{(u,v)\in\mathbb R^2 \middle| u>0,v>0,u^{\frac1a}+v^{\frac1b}<1\right\},$$
et $(U,V)$ est un couple distribué uniformément sur $D$, alors 

$$X = \frac{U^{\frac1a}}{U^{\frac1a}+V^{\frac1b}} \sim \beta(a,b).$$

#### Question 2)
Simuler $(U,V)$, en utilisant une méthode de rejet, avec le fait que $D \subset [0,1]^2$. On poura comparer $X$ avec [`np.random.beta`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.beta.html).


In [ ]:
a = 1.2
b = 1.5

N = 100000

Xref = np.random.beta(a,b,N)

Utilde = np.random.uniform(0,1,N)
Vtilde = np.random.uniform(0,1,N)


## Implémenter ici la méthode de rejet
# U = 
# V = 
# X = U**(1/a) / (U**(1/a) + V**(1/b))

## plot
# fig = plt.figure()

# plt.hist(Xref,density=True,label="numpy",alpha=0.5,color="r",bins=500)
# plt.hist(X,density=True,label="maison",alpha=0.5,color="b",bins=500)

# plt.legend(loc="best")

# plt.show()

### VIII) Simulation d'une loi $\Gamma$

On cherche à implémenter une méthode pour simuler une loi $\Gamma(a,\theta)$, où $a,\theta>0$.
On écrit $ a = \alpha + \lfloor a\rfloor$ avec $0\leq \alpha <1$. Cette écriture est unique.


#### Question 1)
Vérifier (par le calcul) que
$$ \frac1\theta\left(Y - \ln\left(\prod_{k=1}^{\lfloor a\rfloor}U_k\right)\right) \sim \Gamma(a,\theta),$$
où les $U_i$ sont <i>iid</i> uniformes sur $[0,1]$ indépendantes de $Y\sim \Gamma(\alpha,1)$.

#### Vérification numérique
avec [`np.random.gamma`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.gamma.html).

In [ ]:
a = np.pi  # 3.14..
m = 3
alpha = a-3
theta = 0.5

Galpha = gamma(alpha)
Ga = gamma(a)

p = lambda x: x**(a-1)*np.exp(-theta*x)*(theta**a)/Ga

N = 10000  # nombre de tirages
G1 = np.random.gamma(shape=a, scale=1/theta, size=N) # loi gamma avec numpy
G2 = np.random.gamma(shape=alpha, size=N)
U=np.random.uniform(size=(N,m))
G2-=np.sum(np.log(U),axis=1) # log du produit= somme des log
G2 /= theta

Tmax = 20
T = np.linspace(0,Tmax,1000)
fig = plt.figure()
plt.plot(T,p(T),color="black",label="densité")
plt.hist(G1, density=True, bins=100, color="r", alpha=0.5, label="numpy",range=(0,Tmax))
plt.hist(G2, density=True, bins=100, color="b", alpha=0.5, label="maison",range=(0,Tmax))

plt.legend(loc="best")
plt.show()

Le seul problème est donc de simuler $Y\sim\Gamma(\alpha,1)$ pour $0<\alpha<1$.

### Méthode 1: rejet par rapport à une densité bien choisie
On suppose $\alpha>0$. Montrer qu'il est possible de simuler $Y$ par rejet, par rapport à la densité
$$q(x)= \begin{cases} \frac{\alpha e}{\alpha+e}x^{\alpha-1} && 0< x \leq 1,\\
\frac{\alpha e}{\alpha+e} e^{-x} && x>1\end{cases}.$$

Montrer que $k^*=\frac{\alpha + \mathrm{e}}{\alpha\mathrm{e}\Gamma(\alpha)}$.
Expliquer comment échantillonner selon $q$ avec la méthode d'inversion de la fonction de répartition.

In [ ]:
T = np.linspace(0.1, 2, 100)
palpha = lambda x : np.exp(-x)*(x**(alpha-1))/Galpha

k = (alpha+np.exp(1))/(alpha*np.exp(1))
q = lambda x: ((x**(alpha-1))*((0 < x) &(x <=1)) + np.exp(-x)*(x>1))/k

fig = plt.figure()
plt.plot(T, palpha(T), label="p")
plt.plot(T, q(T), label="q")
plt.legend(loc="best")
plt.show()

#### Question 2)
Échantillonner $Y$ un vecteur <i>iid</i> de densité $q$.

In [ ]:
N = 1000
U = np.random.rand(N)
Y = np.zeros_like(U)
# implémenter ici la méthode d'inversion de la fonction de répartition

## plot

# fig = plt.plot()
# plt.hist(Y, density=True, label="histogramme", bins=50)
# plt.plot(x, qx, label="q")
# plt.legend(loc="best")
# plt.ylim(0, 2)
# plt.xlim(0,4)
# plt.show()

#### Question 3)
Implémenter la méthode de rejet

In [ ]:
V = np.random.uniform(size=N)

## implémenter ici le rejet
# G3 = 
# print("k theorique = ", k," k empirique = ",N/G3.size)

In [ ]:
## plot

# G4=np.random.gamma(shape=alpha,scale=1.0,size=N)
# fig = plt.figure()
# plt.hist(G4, density=True, bins=200, color="r", alpha=0.5, label="numpy")
# plt.hist(G3, density=True, bins=200, color="b", alpha=0.5, label="maison")
# plt.ylim(0,2)
# plt.xlim(0,4)
# plt.legend(loc="best")
# plt.show()

### Méthode 2: égalité en loi + rejet
On va maintenant simuler directement une $\Gamma(a,1)$.
On utilise l'égalité en loi 
$$\frac{Y}{X} \overset{\mathcal L}{=} G\sim \Gamma(a,1),$$
où le couple $(X,Y)$ est distribué uniformément sur 
$$ D_a = \left\{(x,y)\in \mathbb{R}_{>0}^2 : 0< x < \sqrt{f\left(\frac{y}{x}\right)}\right\},$$
avec
$$f(z)=z^{a-1}e^{-x}1_{z>0}.$$

#### Question 4)
Calculer $\sup_{z>0}f(z)$, et $\sup_{z>0}z^2f(z)$. En déduire
$$ D_a \subset [0,x_a]\times [0,y_a],\qquad x_a = \left(\frac{a-1}{\mathrm{e}}\right)^{\frac{a-1}2},\,y_a=\left(\frac{a+1}{\mathrm{e}}\right)^{\frac{a+1}2}.$$


In [ ]:
T = np.linspace(0,10,1000)

fT = T**(a-1)*np.exp(-T)
fig = plt.figure()
plt.plot(T,fT,label="f(z)",color="r")
plt.plot(T,T**2*fT,label="z²f(z)",color="b")

xa = np.sqrt(((a-1)/np.e)**(a-1))
ya =np.sqrt(((a+1)/np.e)**(a+1))

plt.axhline(xa**2,linestyle="dashed",color="r")
plt.axhline(ya**2,linestyle="dashed",color="b")
plt.legend(loc="best")
plt.show()

On peut alors simuler $(X,Y)$ uniforme sur $D_a$ par la méthode de rejet, par rapport à la densité uniforme sur 
$$]0,x_a[\times ]0,y_a[,$$
avec $x_a= \left(\frac{a-1}{e}\right)^{\frac{a-1}2}$, et $y_a=\left(\frac{a+1}{e}\right)^{\frac{a+1}2}$.

#### Question 5)
Soit
$$(X,Y)\sim \mathcal U(D_a),$$
c'est-à-dire de densité
$$p_{X,Y}(x,y) = \frac{{\bf{1}}_{D_a}(x,y)}{|D_a|}.$$
où $|D_a|$ est la mesure de Lebesgue de $D_a\subset \R^2$. Calculer la loi de 
$$(X,W) = (X,Y/X),$$
et en **déduire** $|D_a| = \Gamma(a)/2$, puis $Z = W/\theta \sim \Gamma(a,\theta)$.

#### Question 6)
Implémenter cette méthode, par rejet par rapport à une variable $\mathcal U([0,x_a]\times [0,y_a])$.

In [ ]:
Ux = xa * np.random.uniform(size=N)
Uy = ya * np.random.uniform(size=N)

## Implémenter le rejet
# Y = 
# X = 
print("k theorique = ",k," k empirique = ",N/X.size)

In [ ]:
## plot

# G4=np.random.gamma(shape=a,scale=1/theta,size=N)
# G5=(Y/X)/theta

# fig = plt.figure()
# plt.hist(G4, density=True, bins=200, color="r", alpha=0.5, label="numpy")
# plt.hist(G5, density=True, bins=200, color="b", alpha=0.5, label="maison")
# plt.ylim(0,0.25)
# plt.xlim(0,20)
# plt.legend(loc="best")
# plt.show()
